## basics to get everything to work
[Learn how to find your Qualtrics IDs](https://www.qualtrics.com/support/integrations/api-integration/finding-qualtrics-ids/)

In [8]:
# ---- import packages
import pandas as pd
import json
import os
import requests
import base64
import webbrowser

# ---- import qualtrics ids
qualtrics_data_center = 'YOUR QUALTRICS DATA CENTER'
api_token = 'YOUR QUALTRICS API TOKEN'
survey_id = 'YOUR QUALTRICS SURVEY ID'

---
## Let's import the (made up) user data into our notebook

In [9]:
# ---- import user data
user_data = pd.read_excel(os.path.abspath('data/user_data.xlsx'))

# ---- show top five lines of user data
user_data.head()

,email,market,device_type,monthly_active_user,account_type,sessions_in_app_30_days,searches_in_app_30_days
0,example_email0@outlook.com,Canada,android,1,paid,1,1
1,example_email1@icloud.com,US,android,1,free,17,0
2,example_email2@outlook.com,US,android,1,paid,4,4
3,example_email3@icloud.com,Canada,apple,1,paid,2,0
4,example_email4@gmail.com,US,android,0,paid,0,0


---
## To embed user data into a survey, you need to create the data fields in the Qualtrics survey
## Rather than go to the website and type in the column headers by hand, let's do it here

In [10]:
# create embedded data fields in the Qualtrics survey (Qualtrics need to be expect that we are going to send user data)
# ---- reformat user data headers into embedded data fields json
user_data_headers = {'embeddedDataFields': [{'key': column_header} for column_header in user_data.columns]}

# ---- define url for post call
url = 'https://{}/API/v3/surveys/{}/embeddeddatafields'.format(qualtrics_data_center, survey_id)

# ---- post data to survey
requests.post(url, json = user_data_headers, headers={'x-api-token': api_token, 'content-type': 'application/json'})

<Response [200]>

## Let's go to the survey and make sure that it worked!

In [4]:
webbrowser.open('https://duke.ca1.qualtrics.com/Q/EditSection/Blocks?ContextSurveyID=SV_2t0TlwmrGFbCFsp');

---
## Now we create unique survey links. When a person opens the survey using their unique link, their user data gets embedded

In [11]:
# create unique survey links; when a user opens the survey link, it embeds their user data into the survey response
unique_links = []
for row in user_data.index:
    # ---- data needs to be base64 encoded
    encoded_user_data = base64.b64encode(bytes(user_data.loc[row].to_json(), 'utf-8')).decode('utf-8')
    # ---- encoded data gets appended to the end of the survey link using Qualtric's Q_EED function
    unique_links.append('https://{}/jfe/form/{}?Q_EED={}'.format(qualtrics_data_center, survey_id, encoded_user_data))
user_data['unique_survey_link'] = unique_links

# ---- export survey distribution list
email_distribution_list = user_data.set_index('email')[['unique_survey_link']]
email_distribution_list.to_excel(os.path.abspath('data/survey_distribution_list.xlsx.xlsx'))

# ---- show top 5 lines of email distribution list
email_distribution_list.head()

,unique_survey_link
email,
example_email0@outlook.com,https://duke.ca1.qualtrics.com/jfe/form/SV_2t0...
example_email1@icloud.com,https://duke.ca1.qualtrics.com/jfe/form/SV_2t0...
example_email2@outlook.com,https://duke.ca1.qualtrics.com/jfe/form/SV_2t0...
example_email3@icloud.com,https://duke.ca1.qualtrics.com/jfe/form/SV_2t0...
example_email4@gmail.com,https://duke.ca1.qualtrics.com/jfe/form/SV_2t0...


## Let's open the first link and make sure everything worked!

In [12]:
webbrowser.open(email_distribution_list.unique_survey_link.values[0]);